# 3. 텍스트유사도

* 챗봇엔진에서 입력된 문장과 쳇봇시스템에서의 답변이 얼마나 유사한지에 따라 적절한 답변이 가능하다.

## 3.1 n-gram 유사도

* n-gram은 문장에서 n개의 연속적인 단어 시퀀스를 의미한다.
* n-gram은 문장에서 n개의 단어를 token으로 사용한다.
* `이웃한 단어의 출현횟수를 통계적으로 표현해서 텍스트의 유사도를 계산하는 방법`이다.
* 참고 : https://uumini.tistory.com/69

##### 문장간 유사도 계산

* 문장을 n-gram으로 토큰을 분리한 후 단어문서행렬(TDM, Term Document Matrix)를 만든다.
* 이후 `두 문장을 비교해 동일단어의 출현빈도를 확률로 계산해 유사도`를 구할 수 있다.
* A, B 두 문장이 있을 때 B가 A와 얼마나 유사한지 확률을 구하는 수식
>n-gram유사도
>>$simularity=\frac{tf(A,B)}{tokens(A)}$
>* tf(term frequnct)는 두 문장 A와 B에서 동일한 토큰의 출현빈도를 의미
>* tokens는 해당 문장에서 전체 토큰수를 의미
>* 여기서 토큰이란 n-gram으로 분리된 단어 즉, 기준문장 A의 전체 토큰중에서 A와 B의 동일토큰의 확률표현식
>* 1.0에 가까울 수록 B가 A에 유사하다고 볼 수 있다.

##### 2-gram을 이용한 예제
>* A : 6월에 뉴턴은 선생님의 제안으로 트리니티에 입학했다.
>* B : 6월에 뉴턴은 선생님의 제안으로 대학교에 입학했다.
>||(6월, 뉴턴)|(뉴턴,선생님)|(선생님, 제안)|(제안, 트리니티)|(트리니티, 입학)|(입학)||
|:-----|:---------:|:---------:|:---------:|:---------:|:---------:|:---------:|:---------:|
|A|1|1|1|1|1|1|6(tokens(A)|
|B|1|1|1|0|0|1|4(tf(A, B)|
>>* 4/6 = 0.66 즉, 문장 B는 A와 66% 유사하다

In [1]:
sentence1 = '6월에 뉴턴은 선생님의 제안으로 트리니티에 입학하였다'
sentence2 = '6월에 뉴턴은 선생님의 제안으로 대학교에 입학하였다'
sentence3 = '나는 맛잇는 밥을 뉴턴 선생님과 함께 먹었습니다.'

In [2]:
from konlpy.tag import Komoran

# 1. 어절단위 n-gram
def word_ngram(bow, num_gram):
    text = tuple(bow)
    ngrams = [ text[x:x+num_gram] for x in range(0, len(text))]
    return tuple(ngrams)

# 2. 음절 n-gram분석
def phoneme_ngram(bow, num_gram):
    sentence = ' '.join(bow)
    text = tuple(sentence)
    ngrams = [ text[x:x+num_gram] for x in range(0, len(text))]
    return ngrams

# 3. 유사도계산
def similarity(stn1, stn2):
    cnt = 0
    for token in stn1:
        if token in stn2:
            cnt = cnt + 1
    return cnt/len(stn1)

In [5]:
komoran = Komoran()
bow1 = komoran.nouns(sentence1)
bow2 = komoran.nouns(sentence2)
bow3 = komoran.nouns(sentence3)
print(bow1)
print(bow2)
print(bow3)

['6월', '뉴턴', '선생님', '제안', '트리니티', '입학']
['6월', '뉴턴', '선생님', '제안', '대학교', '입학']
['맛', '밥', '뉴턴', '선생', '님과 함께']


In [11]:
stn1 = word_ngram(bow1, 2)
stn2 = word_ngram(bow2, 2)
stn3 = word_ngram(bow3, 2)
print(stn1)
print(stn2)
print(stn3)

(('6월', '뉴턴'), ('뉴턴', '선생님'), ('선생님', '제안'), ('제안', '트리니티'), ('트리니티', '입학'), ('입학',))
(('6월', '뉴턴'), ('뉴턴', '선생님'), ('선생님', '제안'), ('제안', '대학교'), ('대학교', '입학'), ('입학',))
(('맛', '밥'), ('밥', '뉴턴'), ('뉴턴', '선생'), ('선생', '님과 함께'), ('님과 함께',))


In [12]:
r1 = similarity(stn1, stn2)
r2 = similarity(stn1, stn3)
print(r1, r2)

# 결과분석
# n-grams은 문장에 나타나는 단어의 빈도수를 계산하는 것이 아니라 연속된 문장에서의 n개의 단어의 
# 유사도를 계산한다. 
# n-grams의 모델에서 n개의 값의 설정은 매우 중요하다. 보통 1~5사이의 값을 사용한다.

0.6666666666666666 0.0


## 3.2 코사인유사도

* 단어나 문장을 벡터로 표현할 수 있다면 벡터간 거리나 각도를 이용해 유사성을 파악할 수 있다.
* 벡터간의 거리를 구하는 방법은 다양하지만 `코사인유사도 cosine similarity`를 사용
  - 코사인유사도는 `두 벡터 간 코사인각도를 이용해 유사도를 측정하는 방법`이다.
  - 일반적으로 벡터의 크기가 중요하지 않을 때 그 거리를 측정하기 위해 사용ㅎ나다.
  - 예를 들어 단어의 출현빈도를 통해 유사도계산을 한다면 동일단어가 많이 포함될 수록 벡터크기가 커진다.
  - 이때 `코사인유사도는 벡테크기와 상관없이 결과가 안정적`이다.
  - 코사인유사도는 다양한 차원에서 적용이 가능해 실무에서 많이 사용한다.
  
  >코사인유사도 $$similarity = cos\Theta = \frac{A\cdot B}{\left \| A \right \|\left \| B \right \|} = \frac{\sum_{i=1}^{n}A_i\cdot B_i}{\sqrt{\sum_{i=1}^{n}(A_i)^2}\cdot \sqrt{\sum_{i=1}^{n}(B_i)^2}}$$
  
  >##### 2-gram을 이용한 예제
  >* A : 6월에 뉴턴은 선생님의 제안으로 트리니티에 입학했다.
  >* B : 6월에 뉴턴은 선생님의 제안으로 대학교에 입학했다.
  >* 단어문서행렬표현(명사만 추출)
  >|문장|6월|뉴턴|선생님|제안|트리니티|입학|대학||
   |:-----|:-------:|:-------:|:-------:|:-------:|:-------:|:-------:|:-------:|:-------:|
   |A|1|1|1|1|1|1|0|
   |B|1|1|1|0|0|1|1|
  >>* A = [1,1,1,1,1,1,0]
  >>* B = [1,1,1,1,0,1,1]
  >>* ${A\cdot B} = {\sum_{i=1}^{n}A_i\cdot B_i}$
  >>$= (1x1)+(1x1)+(1x1)+(1x1)+(1x0)+(1x1)+(0x1)$
  >>$=1+1+1+1+0+1+0$
  >>$=5$
  >>
  >>* $\left \| A \right \|\left \| B \right \| = \sqrt{\sum_{i=1}^{n}(A_i)^2}\cdot \sqrt{\sum_{i=1}^{n}(B_i)^2}$
  >>$=\sqrt{1^2+1^2+1^2+1^2+1^2+1^2+1^0}x\sqrt{1^2+1^2+1^2+1^2+0^2+1^2+1^2}$
  >>$=\sqrt{6}x\sqrt{6}$
  >>$=\sqrt{36}$
  >>$=6$
  >>
  >>$similarity = cos\Theta = \frac{A\cdot B}{\left \| A \right \|\left \| B \right \|} = \frac{5}{6}$
  >>$=0.8333333$
  
##### 코사인유사도에서는 주로 L2norm(유클리드노름)을 주로 사용
>$유클리드노름 : l2norm = \sqrt{\sum\left | x_i \right |^2}$

* 참고 코사인의 개념 : http://mwultong.blogspot.com/2008/03/cos-cosine-graph.html

In [13]:
# 코사인유사도계산
from konlpy.tag import Komoran
import numpy as np
from numpy import dot
from numpy.linalg import norm

In [29]:
# 코사인유사도계산함수
def cos_sim(vec1, vec2):
    return dot(vec1, vec2) / (norm(vec1) * norm(vec2))

In [30]:
# TDM(Term Document Matrix) : 단어문서행렬
# 비교문장에서 추출한 단어를 기준으로 문장에 해당하는 단어들이 얼마나 포함되어 있는지를
# 나타내는 것이다. 
# 단어문서행렬생성함수
def make_term_doc_mat(sentence_bow, word_dics):
    freq_mat = {}
    
    # 단어문서행렬초기화
    for word in word_dics:
        freq_mat[word] = 0  
        
    # 단어문서행렬생성
    for word in word_dics:
        if word in sentence_bow:
            freq_mat[word] += 1
            
    return freq_mat

# TDM에서 표현된 토큰들의 출현빈도수를 벡터로 변환함수 - 단어벡터생성
def make_vector(tdm):
    vec = []
    for key in tdm:
        vec.append(tdm[key])
    
    return vec

In [31]:
sentence1 = '6월에 뉴턴은 선생님의 제안으로 트리니티에 입학하였다'
sentence2 = '6월에 뉴턴은 선생님의 제안으로 대학교에 입학하였다'
sentence3 = '나는 맛잇는 밥을 뉴턴 선생님과 함께 먹었습니다.'

In [32]:
komoran = Komoran()
bow1 = komoran.nouns(sentence1)
bow2 = komoran.nouns(sentence2)
bow3 = komoran.nouns(sentence3)

In [33]:
# 단어묶음 리스트를 한 개의 리스트로 합치기
bow = bow1 + bow2 + bow3
print(len(bow), bow, '\n')

# 중복제거후 새로운 단어리스트를 생성
word_dics = []
for token in bow:
    if token not in word_dics:
        word_dics.append(token)
        
# 문장별 단어문서행렬 생성
freq_list1 = make_term_doc_mat(bow1, word_dics)
freq_list2 = make_term_doc_mat(bow2, word_dics)
freq_list3 = make_term_doc_mat(bow3, word_dics)
print(freq_list1)
print(freq_list2)
print(freq_list3)

17 ['6월', '뉴턴', '선생님', '제안', '트리니티', '입학', '6월', '뉴턴', '선생님', '제안', '대학교', '입학', '맛', '밥', '뉴턴', '선생', '님과 함께'] 

{'6월': 1, '뉴턴': 1, '선생님': 1, '제안': 1, '트리니티': 1, '입학': 1, '대학교': 0, '맛': 0, '밥': 0, '선생': 0, '님과 함께': 0}
{'6월': 1, '뉴턴': 1, '선생님': 1, '제안': 1, '트리니티': 0, '입학': 1, '대학교': 1, '맛': 0, '밥': 0, '선생': 0, '님과 함께': 0}
{'6월': 0, '뉴턴': 1, '선생님': 0, '제안': 0, '트리니티': 0, '입학': 0, '대학교': 0, '맛': 1, '밥': 1, '선생': 1, '님과 함께': 1}


In [34]:
# 각 문장별로 벡터를 생성해서 넘파이배열로 변환해서 전달 -> 행렬연산을 하기 위해
doc1 = np.array(make_vector(freq_list1))
doc2 = np.array(make_vector(freq_list2))
doc3 = np.array(make_vector(freq_list3))

# 코사인유사도계산
r1 = cos_sim(doc1, doc2)
r2 = cos_sim(doc1, doc3)

print(r1, r2)

# 결과분석
# n-gram(64%)보다 cos유사도(83%)방식이 정확도가 높다.

0.8333333333333335 0.18257418583505536
